In [1]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
import yfinance as yf
import numpy as np

In [2]:
ticker = 'ADVANC.BK'
start_date = '2017-09-14'
end_date = '2023-05-29'

def load_data(ticker):
    try:            
      data = yf.download(ticker, start_date, end_date)
      data.reset_index(inplace=True)
      return data
    except Exception as e:
      print(f'An exception occurred while executing load_data: {e}')

In [3]:
class TrailingStrategy(Strategy):

    __sl_amount = 6.

    def init(self):
        super().init()

    def set_trailing_sl(self, sl_amount: float = 6):

        """
    Set the trailing stop loss as $n below the current price (for long positions)
        Works for future bars only
        """
        self.__sl_amount = sl_amount

    def next(self):
        super().next()
        # Can't use index=-1 because self.__atr is not an Indicator type
        index = len(self.data)-1

        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf,
                               self.data.Close[index] -  self.__sl_amount)
            else:
                trade.sl = min(trade.sl or np.inf,
                               self.data.Close[index] +  self.__sl_amount)
                
my_data = load_data(ticker)

bt = Backtest(my_data, Strategy, cash = 10000, commission=0.002)

stats = bt.run(
        rsi_window = 21,
        lower_bound = 40,
        stop_range = 5,
        )

print(stats)
bt.plot()

[*********************100%***********************]  1 of 1 completed


/var/folders/69/k2882s3d5jbbkmnl3p42wg4ws68910/T/ipykernel_30512/1954349945.py:31: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(my_data, Strategy, cash = 10000, commission=0.002)


TypeError: Can't instantiate abstract class Strategy with abstract methods init, next